# CS524: Introduction to Optimization Lecture 5

#### Students:
- Elizaveta Stepanova
- Neal Desai
- Shangchen (Spencer) Du

#### Lecture date:
- September 13, 2019

#### Link to pdf of lecture slides
- [Lecture 5](https://canvas.wisc.edu/courses/170254/files/folder/Lecture%20slides?preview=9423251)

#### Relevant homework exercises
<b><i>hw2_alloy_blending</i></b> and <i><b>hw2_bussville</i></b>: Both of these problems have optimal solutions that should be solved using a "general" model. This is similar to what is demonstrated in the Diet problem below. The expression of the decision variable will be correct regardless of the size of the sets and table/input data. 


#### References:
1. Santos, Francisco. "A counterexample to the Hirsch conjecture." arXiv preprint arXiv:1006.2814 (2010).   
Link: https://arxiv.org/abs/1006.2814

2. “Simplex Method.” From Wolfram MathWorld.    
Link: http://mathworld.wolfram.com/SimplexMethod.html

3. Reeb, James Edmund, and Scott A. Leavengood. "Using the graphical method to solve linear programs." (1998).    
Link: https://ir.library.oregonstate.edu/concern/open_educational_resources/6q182k541

4. "20 Things You Should Know About Your Nutrition" Live Well with Unity Point Clinic (2015).    
Link: - https://www.unitypoint.org/livewell/article.aspx?id=060291df-8d2b-40ca-81f2-261b7168ce77

### Lecture Outline

#### Last Time
- Simple Input/Output LP
- Sets: Algebraic Modeling

#### Today
- Another I/O model - The Diet Problem.
- What can go wrong in LP?
    - Unbounded / Infeasible / Degenerate LPs

### Data - The Diet Problem
<table>
    <tr>
        <th></th>
        <th>QP</th>
        <th>MD</th>
        <th>BM</th>
        <th>FF</th>
        <th>MC</th>
        <th>FR</th>
        <th>SM</th>
        <th>1M</th>
        <th>OJ</th>
        <th style="color:red">Red'd</th>
    </tr>
    <tr>
        <th style="color:red">Cost</th>
        <th>1.84</th>
        <th>2.19</th>
        <th>1.84</th>
        <th>1.44</th>
        <th>2.99</th>
        <th>0.77</th>
        <th>1.29</th>
        <th>0.6</th>
        <th>0.72</th>
        <th></th>
    </tr>
    <tr>
        <th>Prot</th>
        <th>28</th>
        <th>24</th>
        <th>25</th>
        <th>14</th>
        <th>31</th>
        <th>3</th>
        <th>15</th>
        <th>9</th>
        <th>1</th>
        <th>55</th>
    </tr>
    <tr>
        <th>VitA</th>
        <th>15</th>
        <th>15</th>
        <th>6</th>
        <th>2</th>
        <th>8</th>
        <th>0</th>
        <th>4</th>
        <th>10</th>
        <th>2</th>
        <th>100</th>
    </tr>
    <tr>
        <th>VitC</th>
        <th>6</th>
        <th>10</th>
        <th>2</th>
        <th>0</th>
        <th>15</th>
        <th>15</th>
        <th>0</th>
        <th>4</th>
        <th>120</th>
        <th>100</th>
    </tr>
     <tr>
        <th>Calc</th>
        <th>30</th>
        <th>20</th>
        <th>25</th>
        <th>15</th>
        <th>15</th>
        <th>0</th>
        <th>20</th>
        <th>30</th>
        <th>2</th>
        <th>100</th>
    </tr>
    <tr>
        <th>Iron</th>
        <th>20</th>
        <th>20</th>
        <th>20</th>
        <th>10</th>
        <th>8</th>
        <th>2</th>
        <th>15</th>
        <th>0</th>
        <th>2</th>
        <th>100</th>
    </tr>
    <tr>
        <th>Cals</th>
        <th>510</th>
        <th>370</th>
        <th>500</th>
        <th>370</th>
        <th>400</th>
        <th>220</th>
        <th>345</th>
        <th>110</th>
        <th>80</th>
        <th>2000</th>
    </tr>
    <tr>
        <th>Carb</th>
        <th>34</th>
        <th>33</th>
        <th>42</th>
        <th>38</th>
        <th>42</th>
        <th>26</th>
        <th>27</th>
        <th>12</th>
        <th>20</th>
        <th>350</th>
    </tr>
</table>

When we typically get a problem, we get data that comes in some kind of table. Otherwise, we’ll have to extract data from the problem description that will end up that way. We’ll need row and column indices that will be described as sets. If you can identify sets, it will make your problem easier to solve.

### Quiz

#### Let’s do it
- Can you write an LP that will meet your daily nutritional requirements at a minimum cost?
- How about a "general" model?
    - Let's generate an general (abstract) model. Similar to BigTopBrass trophy example. We abstracted out the notion of the data (trophies and requirements). We made a general model for a particular problem. We’re going to do the same model today!


### Elements of an Optimization
#### Variables
- What are we trying to decide?
- How many of each item to eat.
- Let $x_{j} $: Be the number of item $ j $ to eat.
    - (e.g. $x_{QP}$: Number of quarter pounders).
    
#### Objective
- Minimize cost
- How much does a daily menu cost?

### Costing
- So if I bought my regular lunch: 3 quarter pounders, 2 small fries, and a 1% milk, my cost would be

$$3 \times (1.84) + 2\times(1.44) + 1\times(0.6) = $9.00$$

- A general expression for my cost as a function of my decision on what to buy is

$$1.84x_{QP} + 2.19x_{MD} + 1.84x_{BM} + 1.44x_{FF} + 2.29x_{MC} + 0.77x_{FR} + 1.9x_{SM} + 0.6x_{1M} + 0.72x_{OJ}$$

- This is our <font color=red>linear</font> objective function

### The Diet Problem
- My wife tells me that I need to get 100% of my daily nutritional requirements from eating at McGreasy's
- A general expression for the daily amount of Vitamin A that I get by eating at McGreasy's is
    - I could also eat 50 Filet-O-Fish to get my Vitamin A requirements!

$$15x_{QP} + 15x_{MD} + 6x_{BM} + 2x_{FF} + 8x_{MC} + 4x_{SM} + 10x_{1M} + 2x_{OJ}$$

- In general I need that

$$15x_{QP} + 15x_{MD} + 6x_{BM} + 2x_{FF} + 8x_{MC} + 4x_{SM} + 10x_{1M} + 2x_{OJ} \ge 100$$

- Nutritient facts you might be interested in: [20 Things You Should Know About Nutrition](https://www.unitypoint.org/livewell/article.aspx?id=060291df-8d2b-40ca-81f2-261b7168ce77)

### The Final Model
minimize

$$1.84x_{QP} + 2.19x_{MD} + 1.84x_{BM} + 1.44x_{FF} + 2.29x_{MC} \\+ 0.77x_{FR} + 1.9x_{SM} + 0.6x_{1M} + 0.72x_{OJ}$$

subject to

<font color=red>Protein:</font>
$$28x_{QP} + 2.19x_{MD} + 1.84x_{BM} + 1.44x_{FF} + 2.29x_{MC} \\+  0.77x_{FR} + 1.29x_{SM} + 0.6x_{1M} + 0.72x_{OJ} \ge 55$$  
<font color=red>Vitamin A:</font> 
$$15x_{QP} + 15x_{MD} + 6x_{BM} + 2x_{FF} + 8x_{MC} \\+ 4x_{SM} + 10x_{1M} + 2x_{OJ} \ge 100$$  
<font color=red>Vitamin C:</font>
$$6x_{QP} + 10x_{MD} + 2x_{BM} + 15x_{MC} + 15x_{FR} \\+ 4x_{1M} + 120x_{OJ} \ge 100$$
<font color=red>Calcium:</font>
$$30x_{QP} + 20x_{MD} + 25x_{BM} + 15x_{FF} + 15x_{MC} \\+ 20x_{SM} + 30x_{1M} + 2x_{OJ} \ge 100$$  
<font color=red>Iron: </font>
$$20x_{QP} + 20x_{MD} + 20x_{BM} + 10x_{FF} + 8x_{MC} \\+ 2x_{FR} + 15x_{SM} + 2x_{OJ} \ge 100$$  
<font color=red>Calories:</font>
$$510x_{QP} + 370x_{MD} + 500x_{BM} + 370x_{FF} + 400x_{MC} \\+ 220x_{FR} + 345x_{SM} + 110x_{1M} + 80x_{OJ} \ge 2000$$ 
<font color=red>Carbs:</font>
$$34x_{QP} + 35x_{MD} + 42x_{BM} + 38x_{FF} + 42x_{MC}\\ + 26x_{FR} + 27x_{SM} + 12x_{1M} + 20x_{OJ} \ge 350$$  
$$x_{QP}, x_{MD}, x_{BM}, x_{FF}, x_{MC}, x_{FR}, x_{SM}, x_{1M}, x_{OJ} \ge 0$$  

### The Sets View
#### Set
- $F$: Set of possible foods
- $N$: Set of nutrional requirements

#### Parameter
- $c_j$: Per unit cost of item $j \in F$
- $i$ : Lower Bound on amount of nutrient $i \in N$
- $u_i$ : Upper Bound on amount of nutrient $i \in N$
- $a_{ij}$ : Amount of nutrient $i \in N$ in food $j \in F$

#### Problem
$$
\min \sum_{j \in F} c_jx_j \\
l_i \le \sum_{j \in F}a_{ij}x_j \le u_i \quad \forall i \in N \\
x_j \ge 0 \quad \forall j \in F
$$

In [ ]:
%load_ext gams_magic
import matplotlib.pyplot as plt
import mpl_toolkits
import mpl_toolkits.axisartist as axisartist
import numpy as np
import pandas as pd

def plot_init():
    fig = plt.figure(figsize=(8, 8))
    ax = axisartist.Subplot(fig, 111)
    fig.add_axes(ax)
    ax.axis[:].set_visible(False)

    ax.axis["x"] = ax.new_floating_axis(0, 0)
    ax.axis["x"].set_axisline_style("-|>", size=1.0)
    ax.axis["y"] = ax.new_floating_axis(1, 0)
    ax.axis["y"].set_axisline_style("-|>", size=1.0)
    ax.axis["x"].set_axis_direction("bottom")
    ax.axis["y"].set_axis_direction("left")
    return ax

In [ ]:
%%gams
$title McGreasy Diet Problem

$ontext
A simple diet model.
Data taken from "Optimization Models",
Chapter 1, by Robert Fourer.
$offtext

set food /QP, MD, BM, FF, MC, FR, SM, 1M, OJ/;
set nutr /Prot, VitA, VitC, Calc, Iron, Cals, Carb/;

table a(nutr,food)  per unit nutrients
        QP  MD  BM  FF  MC  FR  SM  1M   OJ
Prot    28  24  25  14  31   3  15   9    1
VitA    15  15   6   2   8   0   4  10    2
VitC     6  10   2   0  15  15   0   4  120
Calc    30  20  25  15  15   0  20  30    2
Iron    20  20  20  10   8   2  15   0    2
Cals   510 370 500 370 400 220 345 110   80
Carb    34  33  42  38  42  26  27  12   20
;

parameter min_nutr(nutr) /Prot 55, VitA 100, VitC 100, Calc 100,
    Iron 100, Cals 2000, Carb 350 /;

parameter cost(food) /QP 1.84, MD 2.19, BM 1.84, FF 1.44, MC 2.29,
FR 0.77, SM 1.29, 1M 0.6, OJ 0.72 /;

free variables
    total_cost  Total Cost of Daily Diet
;
positive variables
    x(food)     Number of each type of food to eat
;


equations
    cost_eqn        Define Objective
    min_nutr_eqn(nutr)    Minimum Daily Requirement
;


min_nutr_eqn(nutr)..
    sum(food,a(nutr,food)*x(food)) =G= min_nutr(nutr) ;

cost_eqn..
    total_cost =E= sum(food,cost(food)*x(food)) ;

model diet1 /cost_eqn, min_nutr_eqn/;
solve diet1 using lp min total_cost;

In [ ]:
%gams_pull -d x
display(x)
x = pd.pivot_table(x[x['level'] != 0], index=['food'], values=['level'])
display(x)

### Tips for Gams:  
- command `$exit` can "ignore the rest of the files".
- use `table a(nutr<, food<)` so we don’t have to list the set domains multiple times. The code will extract out the nutrients based on what is in the table. 

### Algebraic Modeling Languages
- "Mostly" Linear and Integer Modeling Languages
    - Mosel: From FICO. Available on CAE Windows machines
    - OPL: From IBM
    - [GLPK-Mathprog](http://www.gnu.org/software/glpk/)
    - [PuLP](http://code.google.com/p/pulp-or/)
    - [Pyomo](http://www.pyomo.org)
- Nonlinear Modeling Languages
    - [GAMS](https://www.gams.com)
        - GAMS is the biggest language used in economics and operations research. It is also easy to link to python.
    - [LINGO](https://www.lindo.com)
    - [MPL](https://www.maximal-usa.com)
    - [AIMMS](https://www.aimms.com)
    - [AMPL](https://www.ampl.com)

### The Diet Problem in AMPL style

```AMPL
set NUTR; <br/>
set FOOD;
```
---
```AMPL
param cost {FOOD} > 0; <br/>
param f_min {FOOD} >= 0; <br/>
param f_max {j in FOOD} >= f_min[j]; <br/>
param n_min {NUTR} >= 0; <br/>
param n_max {i in NUTR} >= n_min[i]; <br/>
param amt {NUTR,FOOD} >= 0;
```
---
```AMPL
var Buy {j in FOOD} >= f_min[j], <= f_max[j];
```
---
```AMPL
minimize total_cost: sum {j in FOOD} cost[j] * Buy[j]; <br/>
subject to diet {i in NUTR}: <br/>
    n_min[i] <= sum {j in FOOD} amt[i,j] * Buy[j] <= n_max[i];
```

### NEOS
- [NEOS](http://www.neos-server.org/) - <font color=red>N</font>etwork <font color=red>E</font>nabled <font color=red>O</font>ptimization <font color=red>S</font>erver
- If you don't have access to a solver for your particular problem, thanks to the good people of the Wisconsin Institute of Discovery, you can have access to their solvers.

### Graphically Solving LP's
- <font style="color:red">Not</font> for production use, but gives insight into what the algorithm for solving the problem is doing

#### Identify Feasible Region
- Graph each constraint as an inequality
- Note which side is feasible
- Identify the feasible region: The set of all feasible solutions
- Remember to include nonnegativity!

#### "Move" Objective
- Draw parallel "isoprofit" lines. (All points on each line give the same value of the objective function)
- These are points that are orthogonal to the objective function vector
- Optimal point(s) will be on the highest isoprofit line that touches the feasible region

### Observations

- Let's Think About Geometry
If there exists an optimal solution to a LP instance, then there exists an optimal solution that exists at an extreme point of the feasible region.

- <font style="color:red">The simplex method </font> <br />
<font style="color:red">0.</font> Start from an extreme point. <br />
<font style="color:red">1.</font> Find an improving direction d. If none exists, <font style="color:red">STOP.</font> The extreme point is an optimal solution.<br />
<font style="color:red">2.</font> Move along d until you hit a new extreme point. <font style="color:red">Go to 1.</font>

### The Simplex Method
- <font style="color: red">The simplex method</font> is a systematic way in which to do the algebra necessary to do steps 0, 1, and 2.

#### Some Definitions and Facts
- An inequality $a^{T}x \leq b$ is binding at $x$ if $a^{T}x = b$.
- An extreme point is the intersection of at least $n$ inequalities in $\mathbb{R}^{n}$.
- Basis: The indices of the n inequalities that are "binding" at an extreme point solution. (The solution itself is sometimes called a <font style="color: red">basic feasible solution</font>).

### Simplex Method: Algorithm of the century?
- Does the simplex algorithm seem like it would be a good algorithm,much less the “algorithm of the century”?
- Depends on how many extreme points – bases – there are?
- Suppose I have <i>m</i> inequalities and <i>n</i> variables.
- The number of extreme points can be expressed as the combination:

$$
  E(m,n) \leq \binom{m}{n} = \frac{m!}{n!(m-n)!}
$$

#### Scalability: There's big and then there's REALLY BIG
- For instance, if m=1100 and n = 1000:
- We end up with 1422967173622153536429816269821859287617187522528973242276876124026984785611024476217005167336675862280817664301537500761917294126969341076641376 extreme points
- Expressed in scientific notation: $$1.42 \times 10^{143}$$
- To put that into context: $$(\text{The number of subatomic particles in the universe})^2$$

### A+++++++++++++++
- If you can solve the following problem, you will get a very good grade:
    - Given a polytope of dimension <i>n</i> consisting of <i>m</i> inequalities, there is a path between any two extreme points consisting of at most <i>m−n<i> edges.
---
- Before you waste your time, this is known as the <i>Hirsch Conjecture</i>. It is one of the most famous open problems in discrete mathematics.
- It was <font style="color:red">recently disproved</font> by Francisco Santos.
    - The smallest counterexample found to date is a <i>n= 20</i> dimensional polytope with <i>m= 40</i> sides for which the shortest path between 2 vertices is 21.
    - <font style="color:red">Moral:</font>    Geometric intuition for High-dimensional objects can only take us so far...

### The Simplex Method: What can go wrong?
- Simplex method (step 2): Move along <i>d</i> until you hit an extreme point.
- <font style="color:red">What if we don't hit an extreme point?</font>
$$\max x_1 + x_2$$
$$\text{s.t.  } x_1 + 2x_2 \geq 1$$
$$x_1,x_2\geq0$$


- Usually this means you forgot some constraints. Maybe your variable bounds?
- Just because the <font style="color:red">region</font> is unbounded doesn't mean the LP is unbounded. 

In [ ]:
ax = plot_init()

x1 = np.arange(-0.2, 1.3, 0.01)
x2 = (-x1 + 1) / 2
ax.plot(x1, x2, c='b')

ax.fill([0, 1, 2, 2, 0], [0.5, 0, 0, 2, 2], "gray")
plt.axis("equal")

plt.show()

In [ ]:
ax = plot_init()

x1 = np.arange(0, 2, 0.01)
x2 = (-x1 + 1) / 2
ax.plot(x1, x2, c='b')
x2 = (-x1 + 1) / 3
ax.plot(x1, x2, c='b')
x2 = 1/2 * x1 - 1/2
ax.plot(x1, x2, c='b')
x1 = np.ones(200)
x2 = np.arange(-1, 1, 0.01)
ax.plot(x1, x2, c='b')

ax.fill([0, 0, 1], [0, 1/3, 0], "gray")
plt.xlim(-1, 2)
plt.ylim(-1, 1)
plt.axis("equal")
plt.xticks(np.arange(-1, 2, 0.5))
plt.yticks(np.arange(-1, 1.1, 0.5))

plt.show()

### <font style="color:red">I will gladly pay you tuesday</font>
- I <font style="color:red">really</font> like hamburgers
- Let's suppose in the diet problem, I decide to <font style="color:red">maximize</font> the number of hamburgers I eat.
- Let: $$B\subset F$$
$$B = \{QP,MD,BM\}$$
- My new objective is to:
$$ \max \sum_{j \in B} x_b $$

In [ ]:
%%gams
set beef(food) / QP, MD, BM /;

free variable total_beef;

equations
    beef_eqn    Amount of beef I get to eat
;

beef_eqn..
    total_beef =E= sum(beef,x(beef));

model beef1 /min_nutr_eqn, beef_eqn/;
solve beef1 using lp maximizing total_beef;

### Mmmm...beef
- Always check the solver status in the solution report when you aredone running GAMS.
- ```gams
**** SOLVER STATUS     1 NORMAL COMPLETION
**** MODEL STATUS      3 UNBOUNDED
**** OBJECTIVE VALUE               50.0000
```
- Obviously, we haven't the number of hamburgers we're allowed to eat.
- We need to add some constraints to do this!
- Let's imagine that my wife loves me, and she says that I can't eat too many "bad" nutrients.

In [ ]:
%%gams
set bad_nutr(nutr) / Cals, Carb /;

parameter max_nutr(bad_nutr);
max_nutr(bad_nutr) = 2 * min_nutr(bad_nutr);
*display max_nutr;

equations
    max_nutr_req(nutr)  "Helen doesn't want the insurance money yet"
;

max_nutr_req(bad_nutr)..
    sum(food,a(bad_nutr,food)*x(food)) =L= max_nutr(bad_nutr);

model beef2 /min_nutr_eqn, max_nutr_req, beef_eqn/;
solve beef2 using lp maximizing total_beef;

### The Simplex Method: What can go wrong?
- Simplex method step 0: 
    - Start from an extreme point
---
- What if there <i>are</i> no extreme points?
    - This usually means that the feasible region is empty
    - The instance is infeasible
$$P=\{x \in \mathbb{R}^2: x_1 + x_2 \leq 1, x_1 + x+2 \geq 2\}$$
---
- How will we know if an instance is infeasible?
    - "Big-M"..."Two Phase"?
    - The solver will tell us!

### Warning!
- It may be hard to blame one constraint for being infeasible
- When building models for the real world, determining what is “causing” the infeasibility may be tough.
- For instance, who's "fault" is this?
$$x_1 - x_2 \geq 1, x_2 - x_3 \geq 1, -x_1 + x_3 \geq 1$$

### My Wife Loves me
- In the interest of extending my life, my wife has requested that I obey the following constraints:
    - Don't eat more than 3 sandwiches per day
    $$x_{QP}+x_{MD}+x_{BM}+x_{FF}+x_{MC}+x_{SM}\leq3$$
    - Don't drink too much
    $$X_{1M} + X_{OJ} \leq 3$$
    - Only two french fries per day
    $$X_{FF} \leq 2$$
---
- With these constraints, the problem is <font style="color:red">infeasible</font>!

In [ ]:
%%gams

equations
    sandwich_eqn    "Only 3 sandwiches per day"
    drinking_eqn    "Don't drink too much"
*    ff_eqn         "French fries aren't good for you"
;

sandwich_eqn..
    x("QP") + x("MD") + x("BM") + x("FF") + x("MC") + x("SM") =L= 3;

drinking_eqn..
    x("OJ") + x("1M") =L= 3;

x.up("FF") = 2;

option limcol = 0;
option limrow = 100;

model helen /min_nutr_eqn, max_nutr_req, beef_eqn, sandwich_eqn, drinking_eqn /;

solve helen using lp maximizing total_beef;

### Handling Infeasibility
- Our first trick: Introduce slack/surplus variables and try to minimize the slack/surplus.
- Suppose I think that the “too much drinking” constraint is the one causing the problem to be infeasible. 
- <font style="color:red">New decision variable </font>s: Number of extra drinks (over three) that I must drink in order to get a feasible solution
$$X_{1M} + x_{OJ} -s \leq 3, s\geq 0$$
- <font style="color:red">New objective:</font> min s

In [ ]:
%%gams
free variables
    s   Surplus variable for drinking_eqn
;

s.lo = 0;

equations
    drinking_eqn_2    Don't drink too much (count extras)
;

drinking_eqn_2..
         x("OJ") + x("1M") =L= 3 + s;

model mindrink   /min_nutr_eqn, max_nutr_req, beef_eqn, sandwich_eqn,
    drinking_eqn_2 /;

solve mindrink using lp minimizing s;

### The Simplex Method: What else can go wrong?
- Simplex method: Step 2
    - Move along d until you hit another extreme point.
- What if moving in our "improving direction" doesn't take us anywhere? 
$$\max x_1 + x_2$$
$$\text{s.t. } x_1 + 2x_2 \leq 1$$
$$x_1 \leq 1$$
$$x_1 + 3x_2 \leq 1$$
$$2x_1 - 4x_2 \leq 2 $$
$$x1, x2 \geq 0$$


In [ ]:
ax = plot_init()

x1 = np.arange(-0.5, 2, 0.01)
x2 = (-x1 + 1) / 2
ax.plot(x1, x2, c='b')
x2 = (-x1 + 1) / 3
ax.plot(x1, x2, c='b')
x2 = 1/2 * x1 - 1/2
ax.plot(x1, x2, c='b')
x1 = np.ones(200)
x2 = np.arange(-1, 1, 0.01)
ax.plot(x1, x2, c='b')

ax.fill([0, 0, 1], [0, 1/3, 0], "gray")
plt.axis("equal")

plt.show()

### I'm a degenerate
- The previous case is known as the LP being degenerate
- Degeneracy is what happens when more than n inequalities intersect at a point. 
- This doesn’t seem likely to happen, but it does happenin nearly all practical problems.
- This is not really a modeling problem, but it can lead tocomputational difficulties.
- What do solvers do?
    - Perturb the inequalities so they don’t intersect
    - Smallest subscript rule (but they actually don't do this)

### Multiple Optimal Solutions
- What if <i>c</i> is orthogonal to an improving direction <i>d</i>?
- maximize:
$$x_1 + x_2$$
- subject to:
$$x_1+x_2 \leq 3$$
$$x_1, x_2 \geq 0$$
- We get an infinite number of optimal solutions. 

In [ ]:
ax = plot_init()

x1 = np.arange(-0.5, 3.5 , 0.01)
x2 = -x1 + 3
ax.plot(x1, x2, c='b')
ax.fill([0, 0, 3], [0, 3, 0], "gray")
plt.axis("equal")

plt.show()

GAMS allows you to compute stuff after you've solved the problem. Let's see how much Vitamin C I ingested in the last solution

In [ ]:
%%gams
scalar jtlvitc;

jtlvitc = sum(food, a("VitC",food) * x.l(food));

In [ ]:
%gams_pull -d x jtlvitc
display(x,jtlvitc)

In [ ]:
%%gams
s.fx = s.l;

* mindrink.holdfixed = 1;
solve mindrink using lp max total_beef;

In [ ]:
%gams_pull -d s x
display(s,x)

In [ ]:
%gams_reset

In [ ]:
%gams_cleanup -k